In [1]:
import firebase_admin
from firebase_admin import credentials, db, firestore

In [2]:
# Path to your service account key
cred = credentials.Certificate(
    'grassthink-47000-firebase-adminsdk-fbsvc-1a39f3a722.json')
firebase_admin.initialize_app(cred, {
    # For Realtime Database:
    'databaseURL': 'https://grassthink-47000-default-rtdb.firebaseio.com'
})
# Firestore will pick up the same credentials automatically.
fs_client = firestore.client()

In [3]:
from firebase_admin import db


In [4]:
import numpy as np
import pandas as pd

In [5]:
df1 = pd.read_csv("sensor1_normalized_data.csv")
df2 = pd.read_csv("sensor2_normalized_data.csv")

In [6]:
df1['timestamp'] = pd.to_datetime(df1['timestamp'], format='%Y-%m-%d %H:%M:%S')
df2['timestamp'] = pd.to_datetime(df2['timestamp'], format='%Y-%m-%d %H:%M:%S')

In [7]:
# Drop rows where nitrogen, phosphorus, or potassium have 0 values in df1
df1 = df1[df1['nitrogen'] != 0]
df1 = df1[df1['phosphorus'] != 0]
df1 = df1[df1['potassium'] != 0]

# Drop rows where nitrogen, phosphorus, or potassium have 0 values in df2
df2 = df2[df2['nitrogen'] != 0]
df2 = df2[df2['phosphorus'] != 0]
df2 = df2[df2['potassium'] != 0]

In [8]:
print(df1.describe())

                           timestamp      humidity   temperature  \
count                          10170  10171.000000  10171.000000   
mean   2025-02-05 06:03:21.143657728     33.519575     16.845630   
min              2025-01-28 14:07:52     29.200000     14.500000   
25%    2025-01-31 11:42:34.249999872     30.500000     15.800000   
50%       2025-02-02 11:12:53.500000     31.900000     16.700000   
75%              2025-02-08 21:30:29     36.900000     17.700000   
max              2025-02-24 06:02:24     45.400000     24.600000   
std                              NaN      3.688098      1.309801   

       conductivity            ph      nitrogen    phosphorus     potassium  \
count  10171.000000  10171.000000  10171.000000  10171.000000  10171.000000   
mean     401.405368      7.795310     44.875234    150.317078    143.433684   
min      204.000000      3.000000      4.000000     55.000000     47.000000   
25%      380.000000      7.400000     40.000000    140.000000    133.00

In [9]:
df1.to_json("sensor1.json", orient="records", date_format="iso")
df2.to_json("sensor2.json", orient="records", date_format="iso")


In [10]:

def upload_to_realtime(df, node_name):
    """
    Pushes each row of df into Realtime Database under the given node.
    """
    ref = db.reference(node_name)
    for _, row in df.iterrows():
        data = {
            'timestamp':   row['timestamp'].isoformat(),  # make sure it's serializable
            'sensor_id':   row['sensor'],
            'humidity':    row['humidity'],
            'temperature': row['temperature'],
            'conductivity': row['conductivity'],
            'ph':          row['ph'],
            'nitrogen':    row['nitrogen'],
            'phosphorus':  row['phosphorus'],
            'potassium':   row['potassium'],
            'dht_humidity':    row['dht_humidity'],
            'dht_temperature': row['dht_temperature']
        }
        ref.push(data)

# Usage:
# upload_to_realtime(df1, 'sensor1')
upload_to_realtime(df2, 'sensor2')
print("Realtime Database upload complete.")


KeyboardInterrupt: 

In [ ]:
def upload_to_firestore(df, collection_name):
    """
    Writes each row of df as a document in Firestore collection.
    """
    for idx, row in df.iterrows():
        # You can choose to use idx or another field as the doc ID
        doc_ref = fs_client.collection(collection_name).document(str(idx))
        doc_ref.set({
            'timestamp':   row['timestamp'],  # Firestore can store Python datetime directly
            'sensor_id':   row['sensor'],
            'humidity':    row['humidity'],
            'temperature': row['temperature'],
            'conductivity': row['conductivity'],
            'ph':          row['ph'],
            'nitrogen':    row['nitrogen'],
            'phosphorus':  row['phosphorus'],
            'potassium':   row['potassium'],
            'dht_humidity':    row['dht_humidity'],
            'dht_temperature': row['dht_temperature'],
            # you can add metadata too:
            'uploaded_at': firestore.SERVER_TIMESTAMP
        })

# Usage:
# upload_to_firestore(df1, 'sensor1')
upload_to_firestore(df2, 'sensor2')
print("Firestore upload complete.")


In [5]:
df1_json = pd.read_json("sensor1.json")
df2_json = pd.read_json("sensor2.json")

In [6]:
df2_json

,timestamp,sensor,humidity,temperature,conductivity,ph,nitrogen,phosphorus,potassium,dht_humidity,dht_temperature
0,2025-01-28 14:07:52,Sensor2,72.7,26.8,3877,3.0,770,1827,1833,0.0,0.0
1,2025-01-28 14:07:57,Sensor2,73.5,26.8,3877,3.4,770,1827,1833,0.0,0.0
2,2025-01-28 14:08:02,Sensor2,74.2,26.8,3877,6.0,770,1827,1833,0.0,0.0
3,2025-01-28 14:08:08,Sensor2,72.7,26.8,3885,6.0,772,1830,1837,0.0,0.0
4,2025-01-28 14:08:13,Sensor2,73.3,26.8,3885,6.0,772,1830,1837,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
10176,2025-02-24 05:42:24,Sensor2,50.2,18.8,1647,5.8,305,751,749,45.1,20.2
10177,2025-02-24 05:47:24,Sensor2,53.4,18.8,1645,5.8,304,750,748,44.9,20.3
10178,2025-02-24 05:52:24,Sensor2,49.4,18.8,1647,5.8,305,751,749,45.1,20.2
10179,2025-02-24 05:57:24,Sensor2,52.4,18.8,1645,5.8,304,750,748,44.8,20.2


In [7]:
print(df1_json.describe())

                           timestamp      humidity   temperature  \
count                          10170  10171.000000  10171.000000   
mean   2025-02-05 06:03:21.143657728     33.519575     16.845630   
min              2025-01-28 14:07:52     29.200000     14.500000   
25%    2025-01-31 11:42:34.249999872     30.500000     15.800000   
50%       2025-02-02 11:12:53.500000     31.900000     16.700000   
75%              2025-02-08 21:30:29     36.900000     17.700000   
max              2025-02-24 06:02:24     45.400000     24.600000   
std                              NaN      3.688098      1.309801   

       conductivity            ph      nitrogen    phosphorus     potassium  \
count  10171.000000  10171.000000  10171.000000  10171.000000  10171.000000   
mean     401.405368      7.795310     44.875234    150.317078    143.433684   
min      204.000000      3.000000      4.000000     55.000000     47.000000   
25%      380.000000      7.400000     40.000000    140.000000    133.00

In [8]:
df1_json[df1_json['timestamp'].isna()].index

Index([9982], dtype='int64')

In [9]:
df1_json = df1_json.drop(index=9982).reset_index(drop=True)

In [10]:
print(df1_json.describe())

                           timestamp      humidity   temperature  \
count                          10170  10170.000000  10170.000000   
mean   2025-02-05 06:03:21.143657728     33.518850     16.845428   
min              2025-01-28 14:07:52     29.200000     14.500000   
25%    2025-01-31 11:42:34.249999872     30.500000     15.800000   
50%       2025-02-02 11:12:53.500000     31.900000     16.700000   
75%              2025-02-08 21:30:29     36.900000     17.700000   
max              2025-02-24 06:02:24     45.400000     24.600000   
std                              NaN      3.687553      1.309707   

       conductivity            ph      nitrogen    phosphorus     potassium  \
count  10170.000000  10170.000000  10170.000000  10170.000000  10170.000000   
mean     401.396362      7.795231     44.873353    150.312684    143.429302   
min      204.000000      3.000000      4.000000     55.000000     47.000000   
25%      380.000000      7.400000     40.000000    140.000000    133.00

In [11]:
print(df2_json.describe())

                           timestamp      humidity   temperature  \
count                          10181  10181.000000  10181.000000   
mean   2025-02-05 06:26:27.107160320     64.059257     18.647490   
min              2025-01-28 14:07:52     48.800000     15.400000   
25%              2025-01-31 11:42:39     59.200000     17.600000   
50%              2025-02-02 11:13:07     66.100000     18.100000   
75%              2025-02-08 22:01:44     68.400000     20.000000   
max              2025-02-24 06:02:24     80.200000     27.100000   
std                              NaN      6.397818      1.656918   

       conductivity            ph      nitrogen    phosphorus     potassium  \
count  10181.000000  10181.000000  10181.000000  10181.000000  10181.000000   
mean    3103.330125      6.006561    608.728809   1453.155977   1456.708673   
min     1406.000000      3.000000    255.000000    635.000000    632.000000   
25%     2257.000000      5.900000    432.000000   1045.000000   1046.00

In [12]:
df1_json[df1_json['timestamp'].isna()].index

Index([], dtype='int64')

In [13]:
def upload_to_firestore_bulk(df, collection_name, batch_size=500):
    """
    Uploads data to Firestore using batch writes for efficiency.
    """
    batch = fs_client.batch()
    counter = 0

    for idx, row in df.iterrows():
        doc_ref = fs_client.collection(collection_name).document(str(idx))

        doc_data = {
            'timestamp':   row['timestamp'],  # Ensure this is a datetime, not string
            'sensor_id':   row.get('sensor', 'sensor_default'),
            'humidity':    row['humidity'],
            'temperature': row['temperature'],
            'conductivity': row['conductivity'],
            'ph':          row['ph'],
            'nitrogen':    row['nitrogen'],
            'phosphorus':  row['phosphorus'],
            'potassium':   row['potassium'],
            'dht_humidity':    row['dht_humidity'],
            'dht_temperature': row['dht_temperature'],
            'uploaded_at': firestore.SERVER_TIMESTAMP
        }

        batch.set(doc_ref, doc_data)
        counter += 1

        # Commit every batch_size documents
        if counter % batch_size == 0:
            batch.commit()
            batch = fs_client.batch()

    # Commit any remaining docs
    if counter % batch_size != 0:
        batch.commit()

    print(f"Uploaded {counter} documents to collection '{collection_name}'.")





In [14]:
upload_to_firestore_bulk(df1_json, 'sensor1')

Uploaded 10170 documents to collection 'sensor1'.


In [15]:
upload_to_firestore_bulk(df2_json, 'sensor2')

RetryError: Timeout of 60.0s exceeded, last exception: 429 Quota exceeded.